## 0. Imports

In [642]:
import pandas as pd
import re

## 1. Data preparation and cleaning

In [643]:
df = pd.read_excel('movies.xlsx')
pd.set_option('display.max_columns', None)

# Truncate
pd.set_option('display.max_rows', False)

# Dont truncate
# pd.set_option('display.max_rows', None)

### Dataset info

In [644]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1402 entries, 0 to 1401
Data columns (total 31 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Film                                     1402 non-null   object 
 1   Year                                     1402 non-null   int64  
 2   Script Type                              1402 non-null   object 
 3   Rotten Tomatoes  critics                 1401 non-null   object 
 4   Metacritic  critics                      1402 non-null   object 
 5   Average critics                          1402 non-null   object 
 6   Rotten Tomatoes Audience                 1401 non-null   float64
 7   Metacritic Audience                      1402 non-null   object 
 8   Rotten Tomatoes vs Metacritic  deviance  1402 non-null   object 
 9   Average audience                         1402 non-null   object 
 10  Audience vs Critics deviance             1402 no

In [645]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Year,1402.0,2012.140514,3.190291,2007.0,2009.0000,2012.000,2015.0000,2017.00
Rotten Tomatoes Audience,1401.0,62.122769,17.435737,17.0,49.0000,62.000,76.0000,98.00
Opening weekend ($million),1402.0,23.200357,28.547541,0.0,6.6025,14.200,28.5975,247.97
Domestic gross ($million),1402.0,74.058024,87.389737,0.0,21.0900,44.045,92.5900,936.66
Worldwide Gross ($million),1402.0,173.008302,233.859025,0.0,38.0000,88.690,208.7500,2781.50
Distributor,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
IMDb Rating,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
IMDB vs RT disparity,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [646]:
df.describe(include='object').T

,count,unique,top,freq
Film,1402,1396,Big Miracle,2
Script Type,1402,17,original screenplay,546
Rotten Tomatoes critics,1401,102,93,29
Metacritic critics,1402,90,51,39
Average critics,1402,196,64,25
Metacritic Audience,1402,78,66,50
Rotten Tomatoes vs Metacritic deviance,1402,82,-1,58
Average audience,1402,116,58,45
Audience vs Critics deviance,1402,78,-5,54
...,...,...,...,...


### Handle missing values

In [647]:
missing_data = df.isnull().sum()
missing_percentage = (missing_data[missing_data > 0] / df.shape[0]) * 100

In [648]:
print('Missing data\n--------------')
print(missing_data)


Missing data
--------------
Film                                          0
Year                                          0
Script Type                                   0
Rotten Tomatoes  critics                      1
Metacritic  critics                           0
Average critics                               0
Rotten Tomatoes Audience                      1
Metacritic Audience                           0
Rotten Tomatoes vs Metacritic  deviance       0
Average audience                              0
Audience vs Critics deviance                  0
Primary Genre                              1383
                                           ... 
Worldwide Gross                               0
Worldwide Gross ($million)                    0
 of Gross earned abroad                       0
Budget ($million)                             0
 Budget recovered                             0
 Budget recovered opening weekend             0
Distributor                                1402
IMDb Rating 

In [649]:
print('Missing data percentages\n--------------')
print(missing_percentage)

Missing data percentages
--------------
Rotten Tomatoes  critics       0.071327
Rotten Tomatoes Audience       0.071327
Primary Genre                 98.644793
Genre                          0.071327
Distributor                  100.000000
IMDb Rating                  100.000000
IMDB vs RT disparity         100.000000
Oscar Winners                 96.005706
Oscar Detail                  96.005706
dtype: float64


In [650]:
df = df.dropna(subset=['Genre', 'Rotten Tomatoes  critics', 'Rotten Tomatoes Audience '])

### Drop any lines with `-` values

In [651]:
df = df[~df.isin(['-']).any(axis=1)]
df[(df == '-').any(axis=1)]

,Film,Year,Script Type,Rotten Tomatoes critics,Metacritic critics,Average critics,Rotten Tomatoes Audience,Metacritic Audience,Rotten Tomatoes vs Metacritic deviance,Average audience,Audience vs Critics deviance,Primary Genre,Genre,Opening Weekend,Opening weekend ($million),Domestic Gross,Domestic gross ($million),Foreign Gross ($million),Foreign Gross,Worldwide Gross,Worldwide Gross ($million),of Gross earned abroad,Budget ($million),Budget recovered,Budget recovered opening weekend,Distributor,IMDb Rating,IMDB vs RT disparity,Release Date (US),Oscar Winners,Oscar Detail


In [652]:
missing_data = df.isnull().sum()
missing_data

Film                                          0
Year                                          0
Script Type                                   0
Rotten Tomatoes  critics                      0
Metacritic  critics                           0
Average critics                               0
Rotten Tomatoes Audience                      0
Metacritic Audience                           0
Rotten Tomatoes vs Metacritic  deviance       0
Average audience                              0
Audience vs Critics deviance                  0
Primary Genre                              1372
                                           ... 
Worldwide Gross                               0
Worldwide Gross ($million)                    0
 of Gross earned abroad                       0
Budget ($million)                             0
 Budget recovered                             0
 Budget recovered opening weekend             0
Distributor                                1391
IMDb Rating                             

### Check for dublicates

In [653]:
df.duplicated(keep=False).sum()

0

### Drop empty columns

In [654]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Year,1391.0,2012.109274,3.177868,2007.0,2009.000,2012.00,2015.000,2017.00
Rotten Tomatoes Audience,1391.0,62.048167,17.428369,17.0,49.000,62.00,76.000,98.00
Opening weekend ($million),1391.0,23.327541,28.607251,0.0,6.755,14.30,28.720,247.97
Domestic gross ($million),1391.0,74.500093,87.569797,0.0,21.440,44.34,93.455,936.66
Worldwide Gross ($million),1391.0,173.782559,234.428996,0.0,38.410,89.00,210.035,2781.50
Distributor,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
IMDb Rating,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
IMDB vs RT disparity,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [655]:
df = df.drop(['Distributor', 'IMDb Rating', 'IMDB vs RT disparity'], axis=1)
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Year,1391.0,2012.109274,3.177868,2007.0,2009.000,2012.00,2015.000,2017.00
Rotten Tomatoes Audience,1391.0,62.048167,17.428369,17.0,49.000,62.00,76.000,98.00
Opening weekend ($million),1391.0,23.327541,28.607251,0.0,6.755,14.30,28.720,247.97
Domestic gross ($million),1391.0,74.500093,87.569797,0.0,21.440,44.34,93.455,936.66
Worldwide Gross ($million),1391.0,173.782559,234.428996,0.0,38.410,89.00,210.035,2781.50


### Drop unecessary columns for classification

In [656]:
df = df.drop(['Opening Weekend', 'Domestic Gross', 'Foreign Gross', 'Worldwide Gross', 'Film', 'Year', 'Release Date (US)', 'Oscar Detail'], axis=1)

### Convert `script type` and `genre` to categorical values

In [657]:
df['Script Type'] = df['Script Type'].str.lower()

In [658]:
script_types = df['Script Type'].unique()

In [659]:
# Split strings and remove trailing spaces
types = []
for str in script_types:
    types.append(re.split(' / |, ',str))
script_types = [j.strip() for sub in types for j in sub]
#Convert to set to remove dublicates
script_types = set(script_types)

In [660]:
# Insert columns for each coresponding script type
for i, item in enumerate(script_types):
    df.insert(i+1, 'st_'+item, 0)

In [661]:
# Assign 1 to each corresponding script type
for i in df.index:
    for script in script_types:
        if script in df.at[i, 'Script Type']:
            df.at[i, 'st_'+script] = 1

In [662]:
df = df.drop(['Script Type'], axis=1)

In [664]:
df = df.drop(['Primary Genre'], axis=1)

In [665]:
# Convert genres to lower to avoid possible dublication
df['Genre'] = df['Genre'].str.lower()
genres = df['Genre'].unique()

In [666]:
# Split strings and remove trailing spaces
unique_genres = []
for str in genres:
    unique_genres.append(re.split(', |. ',str))
genre_types = [j.strip() for sub in unique_genres for j in sub]
#Convert to set to remove dublicates

genre_types = set(genre_types)

In [667]:
# Remove invalid values
genre_types.remove('')
genre_types.remove('adventur')
genre_types.remove('biograph')
genre_types.remove('comed')
genre_types.remove('crim')
genre_types.remove('documentar')
genre_types.remove('dram')
genre_types.remove('famiily')
genre_types.remove('famil')
genre_types.remove('fantas')
genre_types.remove('horro')
genre_types.remove('musi')
genre_types.remove('music')
genre_types.remove('myster')
genre_types.remove('romanti')
genre_types.remove('sci-f')
genre_types.remove('sport')
genre_types.remove('thriler')
genre_types.remove('thrille')

In [668]:
genre_types

{'action',
 'adventure',
 'animation',
 'biography',
 'comedy',
 'crime',
 'documentary',
 'drama',
 'family',
 'fantasy',
 'history',
 'horror',
 'musical',
 'mystery',
 'period',
 'romance',
 'sci-fi',
 'sports',
 'thriller',
 'war',
 'western'}

In [669]:
# Insert columns for each coresponding script type
for i, item in enumerate(genre_types):
    df.insert(i+16, 'gen_'+item, 0)

In [670]:
# Assign 1 to each corresponding script type
for i in df.index:
    for genre in genre_types:
        if genre in df.at[i, 'Genre']:
            df.at[i, 'gen_'+genre] = 1

In [672]:
df = df.drop(['Genre'], axis=1)

In [673]:
df

,st_remake,st_adaptation,st_documentary,st_reboot,st_prequel,st_original screenplay,st_based on a true story,st_sequel,Rotten Tomatoes critics,Metacritic critics,Average critics,Rotten Tomatoes Audience,Metacritic Audience,Rotten Tomatoes vs Metacritic deviance,Average audience,Audience vs Critics deviance,gen_thriller,gen_war,gen_sci-fi,gen_animation,gen_comedy,gen_biography,gen_western,gen_fantasy,gen_documentary,gen_adventure,gen_family,gen_history,gen_action,gen_mystery,gen_drama,gen_period,gen_crime,gen_musical,gen_sports,gen_romance,gen_horror,Opening weekend ($million),Domestic gross ($million),Foreign Gross ($million),Worldwide Gross ($million),of Gross earned abroad,Budget ($million),Budget recovered,Budget recovered opening weekend,Oscar Winners
0,0,1,0,0,0,0,0,0,60,51,56,89.0,71,18,80,-25,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,70.89,210.61,245.45,456.00,53.82%,65,701.64%,109.05%,NaN
1,1,0,0,0,0,0,0,0,88,76,82,86.0,73,13,80,3,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,14.04,53.61,16.17,70.00,23.18%,50,139.56%,28.07%,NaN
2,0,1,0,0,0,0,0,0,50,53,52,56.0,65,-9,61,-9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,15.95,39.57,35.53,75.00,47.31%,32,234.67%,49.85%,NaN
3,0,0,0,0,0,1,0,0,54,56,55,82.0,73,9,78,-23,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,3.82,24.34,5.02,29.00,17.11%,45,65.26%,8.50%,NaN
4,0,0,0,0,0,0,0,1,14,29,22,31.0,45,-14,38,-17,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,10.06,41.80,87.09,129.00,67.57%,40,322.21%,25.15%,NaN
5,0,1,0,0,0,0,0,0,26,39,33,65.0,55,10,60,-28,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,44.31,217.33,143.14,360.00,39.71%,70,514.95%,63.30%,NaN
6,0,0,0,0,0,0,1,0,80,76,78,87.0,75,12,81,-3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,43.57,130.16,137.74,268.00,51.41%,100,267.90%,43.57%,NaN
7,0,0,0,0,0,1,0,0,52,54,53,53.0,64,-11,59,-6,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,38.02,126.63,160.49,287.00,55.90%,150,191.42%,25.35%,NaN
8,0,0,0,0,0,1,0,0,71,59,65,50.0,63,-13,57,9,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,27.52,82.20,114.07,196.00,58.12%,150,130.84%,18.34%,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
